# Récursion
On peut résumer les avantages et inconvénients de la formulation récursive d'un algorithme (qui peut aussi bien trouver une formulation itérative) comme suit:

Avantages:
* Code plus compact, plus simple, plus élégant,
* Code souvent plus facile à débugger une fois compris,
* Traitement des *edges cases* souvent plus simple.
Inconvénients: 
* Par rapport à formulation itérative, chaque appel de fonction dans la formulation récursive ajoute une *frame* à la *call stack* (complexité spatiale en $O(n)$) ce qui peut poser problème dans les cas de *deep recursion* (*stack overflow error*),
* Suivant le language utilisé, le code récursif n'est pas toujours optimisé (pas de *tail call optimization* par exemple),
* Formulations récursives pas forcément triviales au premier abord, peut sembler difficile à comprendre à des utilisateurs peu familiers du raisonnement récursif.  

## Optimisation d'algorithmes récursifs
Au choix d'une formulation récursive d'un algorithme s'associe une complexité temporelle et spatiale (empreinte mémoire). On cite ici deux techniques d'optimisation d'algorithmes récursifs: 
* *Tail-call optimisation*: Il s'agit d'une optimisation du code faite par le compilateur pour des fonctions récursives d'une forme particulière (*tail-recursive functions*). Son but est de diminuer l'empreinte mémoire de l'algorithme sans en changer sa complexité temporelle: pour un même nombre d'appels, l'empreinte mémoire (sur la *call stack*) de l'algorithme sera constante ($O(1)$) au lieu d'être linéaire ($O(n)$).
* Mémoïsation: Il s'agit d'une optimisation dans l'implémentation de l'algorithme (qui se trouve d'appartenir à la famille des problèmes de programmation dynamique). Son but est en stockant des résultats intermédiaires, de réduire la complexité temporelle au prix d'une empreinte mémoire accrue.

Remarque: La *tail recursion* et la mémoïsation sont en fait des *design patterns* de programmation fonctionnelle.

### *Tail-call optimisation* (TCO)
Lorsqu'une fonction est appelée:
* De l'espace lui est alloué sur la *call stack*. C'est dans cet espace que seront stockés ses arguments et ses variables locales.
* Les arguments de la fonction sont copiés dans l'espace alloué. On dit qu'on ajoute une *frame* à la *call stack*
* Le contrôle est passé à la fonction appelée.
* La fonction appelée s'exécute.
* La valeur retournée par la fonction est stockée.
* La *frame* est supprimée, on remonte d'un cran sur la *stack* au niveau de la fonction (*calling function*) qui avait appelé la fonction venant d'être exécuté (*called function*). 
* Le contrôle repasse à la fonction appelante.

#### Implémentation de la TCO
Mettre une fonction sous une forme *tail-recursive* peut (mais pas toujours) demander de faire appel à l'introduction d'une variable appelée *accumulator* qui permet de propager des résultats intermédiaires le long de la *call chain*. La fonction *tail-recursive* prend l'*accumulator* comme argument (ce qui permet de le propager), celui-ci étant initialisé avec une valeur par défaut. 

Comme dit plus haut, la TCO est une optimisation faite au niveau du compilateur. Profiter des gains de la TCO présupposent donc que l'implémentation du language utilisé inclut cette optimisation. Ce n'est par exemple pas le cas de Python qui **n'implémente pas** la TCO. Dans le cas de tels languages, il peut être préférable de privilégier une approche itérative plutôt que récursive.

## Passer d'une fonction récursive à une forme itérative : le *recursion unrolling*
Pour réaliser l'*unrolling* d'une fonction à l'aide d'une boucle while, il faut commencer par la mettre sous sa forme *tail recursive*. Parce que la fonction a pu se mettre sous cette forme, on n'a en théorie lors d'appels récursifs, plus besoin de stocker l'état de la fonction appelée: puisque la fonction ne fait que retourner le résultat de l'appel récursif, on a pas besoin de stocker son environnement local car elle n'en a plus besoin. C'est sur cette observation que reposent des stratégies de *tail call elimination* des languages qui l'implémentent. C'est aussi sur celle-ci que se va s'appuyer l'*unrolling* de la fonction qui va permettre de passer d'une forme récursive à une forme itérative. 

Remarque: L'*unrolling* à l'aide d'une *stack* n'est pas la seule façon de passer d'une fonction récursive à une fonction itérative. Cf. Tabulation.

Sous forme *tail recursive*, la fonction est globalement de la forme:
```python
def func(args, accumulator):
    if stop_condition(args):
        return accumulator
    # function logic
    # update args and accumulator
    return func(args, accumulator)
```
L'unrolling va consister à la mettre sous la forme: 
```python
def unrolled_func(args):
    # initialize variables used to store arguments and accumulator values (called `stack`)
    while condition(stack):
        # function logic
        # update accumulator
        if stop_condition(args):
            # update stack, if not udpated, should must break the loop
    return accumulator
```
Le stockage et la mise à jour des valeurs des arguments n'a pas forcément besoin de faire appel à l'utilisation explicite d'une *data structure*. 

Par exemple pour la factorielle:

In [36]:
def factorial(n):
    if not n:
        return 1
    return n * factorial(n=n-1)

# Tail call transformation
def tail_call_factorial(n, accumulator=1):
    if n==0:
        return accumulator
    return tail_call_factorial(n=n-1, accumulator=accumulator*n)

*Unrolling* sans faire appel à une structure de données:

In [39]:
def implicit_stack_factorial(n):
    accumulator = 1
    i = n # not mandatory but for clarity purposes
    while i:
        accumulator *= i
        i -= 1
    return accumulator

On peut choisir de faire apparaître plus explicitement la condition d'arrêt / le *base case*:

In [ ]:
def implicit_stack_factorial(n):
    accumulator = 1
    i = n # not mandatory but for clarity purposes
    while True:
        if i==0:
            break
        accumulator *= i
        i -= 1
    return accumulator

*Unrolling* sans faire appel à une structure de données (en général une *stack* objet qui en Python est souvent modélisé avec une liste). La condition d'arrêt / le *base case* peut être moins explicite car l'action associée ne va simplement pas updater la *stack* ce qui va avoir pour conséquence d'interrompre le `while` (on a plus le très explicite `break` *statement*).

In [41]:
def explicit_stack_factorial(n):
    stack = [(n, 1)]
    while stack:
        i, accumulator = stack.pop()
        accumulator *= i
        i -= 1
        if i: 
            stack.append((i, accumulator))
    return accumulator

On peut évidemment aboutir après refactorisation à des formes mixtes où seulement une partie des variables nécessaires sont dans stockées dans une structure de données.

## Optimiser la récursion : la mémoïsation
On va ici s'appuyer sur le calcul des termes de la suite de Fibonacci : 
$u_n = u_{n-1} + u_{n-2}$ avec $u_{0} = 0$ et $u_{1} = 1$

In [21]:
# Naive Fibonacci
def fibonacci(n):
    if n in {0, 1}:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

La mémoisation revient à ajouter un comportement à la fonction récursive `f`: avant de procéder au calcul (à l'appel de `f`), on vérifie dans le cache si la valeur n'a pas déjà été calculée. Si c'est le cas, on ne fait que retourner la valeur correspondante, sinon on la calcule, l'ajoute au cache et la retourne. 

On comprend immédiatemment qu'utiliser le *design pattern decorator* est une façon naturelle d'implémenter la mémoïsation. La fonction devant avoir accès au cache, celui-ci doit soit lui être injecté, soit se trouver dans l'*enclosing namespace*: 
* L'injection pose plusieurs problèmes. On ne peut notamment pas utiliser cette technique en dehors de récursions simples comme lors du calcul d'une facorielle. Dans le cas de la suite de Fibonacci où l'appel récursif est `f(n-1) + f(n-2)`, comment passer au second appel de `f` le cache updaté par le premier ? L'injection pose aussi le problème de l'initialisation du cache. Comme pour les *accumulators* on pourrait l'initialiser avec la valeur par défaut d'un argument. Le cache étant cependant mutable, ce n'est pas acceptable en Python.
* La solution est que le cache reste une variable libre de la fonction que cette dernière ira récupérer dans un *enclosing namespace*. Mettre le cache dans le *global namespace* n'est pas acceptable étant entendu que les variables globales sont supposées être constantes sur l'ensemble de la vie du programme. L'implémentation peut alors prendre deux formes (qui sont chacune une variante du *decorator*): l'utilisation d'une *closure* ou d'un objet encapsulant la fonction à mémoïser. 

In [45]:
def memoize(f):
    cache = dict()
    def decorated_func(x):
        if x not in cache:
            cache[x] = f(x)
        return cache[x]
    return decorated_func

@memoize
def fibonacci(n):
    if n in {0, 1}:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [33]:
fibonacci(10)

55

In [44]:
class Memoize:
    def __init__(self, func):
        self.func = func
        self.cache = dict()
        
    def __call__(self, x):
        if x not in self.cache:
            self.cache[x] = self.func(x)
        return self.cache[x]
    
@Memoize
def fibonacci(n):
    if n in {0, 1}:
        return n
    return fibonacci(n-1) + fibonacci(n-2)

In [35]:
fibonacci(10)

55

Dans le cas de la suite de Fibonacci, le gain apporté par la mémoïsation est très important puisqu'on passe d'un calcul en $O(2^n)$ pour la forme naïve à $O(n)$ pour la forme mémoïsée. 

## Passer d'une fonction récursive à une forme itérative : la tabulation

In [43]:
# Tabulated (iterative) Fibonacci
def tab_fibonacci(n):
    table = [0, 1]
    if n > 1:
        for i in range(2, n+1):
            table.append(table[i-1]+table[i-2])
    return table[n]

## Programmation dynamique et récursion
La programmation dynamique (*dynamic programming*) est une approche, un paradigme de résolution de problèmes d'optimisations. L'idée générale de la programmation dynamique est de résoudre un problème initial complexe en résolvant plusieurs sous-problèmes plus simples. Ce raisonnement s'appliquant souvent au sous-problèmes eux-mêmes, la programmation dynamique se préoccupe donc principalement de problèmes ayant une nature récursive. La stratégie d'implémentation et d'optimisation d'un algorithme de programmation dynamique va souvent chercher à s'appuyer sur deux propriétés du problèmes: 
* *Overlapping subproblems* : Cette propriété désigne le fait que des sous-problèmes apparaissent plusieurs fois dans l'arbre de récursion de la résolution du problème. Tous les problèmes de programmation dynamique n'ont pas cette propriété: tous les sous-problèmes peuvent être distincts.
* *Optimal substructure* : Cette propriété désigne le fait que la solution optimale du problème peut être dérivée des solutions optimales de ses différents sous-problèmes. On peut ainsi s'appuyer sur cette propriété en calculant les solutions de chacun des sous-problèmes puis "remonter" à la solution du problème initial. C'est cette propriété qui rend équivalente résolution du problème global et résolution de l'ensemble des sous-problèmes.

On distingue deux approches pour l'implémentation d'algorithmes de résolution de problèmes de programmation dynamique: 
* La mémoïsation (*memoization*): S'appuyant sur la propriété des *overlapping subproblems*, cette stratégie consiste simplement à stocker (mémoriser) le résultat du calcul de chacun des sous-problèmes afin de ne les calculer qu'une fois. C'est une approche *top-down*: on démarre du problème initial et on résout (résursivement) les sous-problèmes correspondant en ne calculant un problème donné qu'une seule fois.
* La tabulation: Cette stratégie consiste simplement à calculer l'ensemble des sous-problèmes puis de "remonter" à la solution du problème initial (approche *bottom-up*).

La tabulation peut être plus avantageuse dans le cas où on a pas d'*overlapping subproblems* et où la mémoïsation n'apporte que peu de gains. D'un autre côté, la mémoïsation a l'avantage par rapport à la tabulation de ne calculer que les sous-problèmes finalement requis là où la tabulation calcule l'ensemble des sous-problèmes possibles. La mémoïsation étant liée à une formulation récursive du problème, le code associé peut s'en trouver plus difficile d'accès.